In [1]:
import numpy as np
import pandas as pd

## CHARGEMENTS DES DONNEES

In [3]:
# Chargement des DataSets
fps_df = pd.read_csv('../../../Data/Regression/fps in video games/fps-in-video-games_clean.csv')
heart_df = pd.read_csv('../../../Data/Classification/Indicators of Heart Disease/heart_2022_no_nans_clean.csv')
retail_df = pd.read_csv('../../../Data/Clustering/retail Data/retail_data_clean.csv')

/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_5981/3500090452.py:2: DtypeWarning: Columns (25,30) have mixed types. Specify dtype option on import or set low_memory=False.
  fps_df = pd.read_csv('../../../Data/Regression/fps in video games/fps-in-video-games_clean.csv')


## SUPPRESSION DES DONNEES

In [4]:
def introduce_missing_values_mcar(df, missing_percentages):
    # Vérifie que les pourcentages sont valides
    if not all(0 <= p <= 1 for p in missing_percentages):
        raise ValueError("Les pourcentages doivent être entre 0 et 1.")

    df_dict = {}

    for percentage in missing_percentages:
        df_missing = df.copy()
        # Pour chaque colonne du DataFrame
        for col in df.columns:
            # Calcule le nombre de valeurs à remplacer par NaN
            n_missing = int(np.floor(percentage * df.shape[0]))
            # Sélectionne aléatoirement les index des valeurs à remplacer
            missing_indices = np.random.choice(df.index, n_missing, replace=False)
            # Remplace les valeurs par NaN
            df_missing.loc[missing_indices, col] = np.nan

        df_dict[f"{int(percentage * 100)}%"] = df_missing

    return df_dict

In [5]:
# Liste des pourcentages de valeurs manquantes à introduire
missing_percentages = [0.0, 0.02, 0.05, 0.07, 0.1,  0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6]

# Génération des DataFrames avec valeurs manquantes
fps_df_with_missing = introduce_missing_values_mcar(fps_df, missing_percentages)
heart_df_with_missing = introduce_missing_values_mcar(heart_df, missing_percentages)
retail_df_with_missing = introduce_missing_values_mcar(retail_df, missing_percentages)

/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_5981/4019777069.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_missing.loc[missing_indices, col] = np.nan
/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_5981/4019777069.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_missing.loc[missing_indices, col] = np.nan
/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_5981/4019777069.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first

In [6]:
# Enregistrement des DataFrames générés
for key, value in fps_df_with_missing.items():
    value.to_csv(f"../../../Data/Regression/fps in video games/Completeness/NaN/fps-in-video-games_{key}.csv", index=False)

for key, value in heart_df_with_missing.items():
    value.to_csv(f"../../../Data/Classification/Indicators of Heart Disease/Completeness/NaN/heart_2022_{key}.csv", index=False)

for key, value in retail_df_with_missing.items():
    value.to_csv(f"../../../Data/Clustering/retail Data/Completeness/NaN/retail_{key}.csv", index=False)